In [ ]:
!pip install mediapipe
#for diplegia
import cv2
import numpy as np
import math
import mediapipe as mp
import csv
import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Functions required

# Distance of a vector p1 from origin, p1 being a tuple
def length(p1):
    return math.sqrt(p1[0]**2 +p1[1]**2+p1[2]**2)

# Minimum distance between two points, point1 and point2, both being tuple
def calculate_distance(point1, point2):
    return math.sqrt(sum((a - b) ** 2 for a, b in zip(point1, point2)))

# Cross product of two vectors p1 and p2
def cross_p(p1,p2):
    return (p1[1]*p2[2]-p1[2]*p1[1],p1[2]*p2[0]-p1[0]*p2[2],p1[0]*p2[1]-p1[1]*p2[0])

# If we have three points Xa,Xb and Xc, And we have to find perpendicular distance from point Xb to XaXc, Xa,Xb,Xc being tuple
def pd_distance(Xa,Xb,Xc):
   l1 = tuple(a - b for a, b in zip(Xa, Xb))
   l2 = tuple(a - b for a, b in zip(Xa, Xc))
   return length(cross_p(l1, l2)) / length(l2)

# Angle between two vectors X1-X2 and X3-X4,
def calculate_angle(X1, X2, X3, X4):
    v1 = np.array(X1) - np.array(X2)
    v2 = np.array(X3) - np.array(X4)
    dot_product = np.dot(v1, v2)
    magnitude_v1 = np.linalg.norm(v1)
    magnitude_v2 = np.linalg.norm(v2)
    angle_radians = np.arccos(dot_product / (magnitude_v1 * magnitude_v2))
    angle_degrees = np.degrees(angle_radians)
    return angle_degrees



In [ ]:
# Importing holistic class from MediaPipe
mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic(static_image_mode=False)

# Manually specify the paths
video_dir = '/content/drive/MyDrive/surge/Journal Dataset/diplegia'
output_dir = '/content/drive/MyDrive/surge/Frameworks/mediapipe/Everyvideoexceldata_mp/dip'

# Ensure the paths are absolute
video_dir = os.path.abspath(video_dir)
output_dir = os.path.abspath(output_dir)

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for video_file in os.listdir(video_dir):
    if video_file.endswith(('.mp4', '.avi', '.mov')):  # Add other video file extensions if needed
        video_path = os.path.join(video_dir, video_file)
        cap = cv2.VideoCapture(video_path)

        csv_file_path = os.path.join(output_dir, f"{os.path.splitext(video_file)[0]}.csv")

        with open(csv_file_path, 'w', newline='') as csvfile:
            num_points = 17
            fieldnames = ['Frame', 'Head', 'Left Shoulder', 'Right Shoulder', 'Left Elbow', 'Right Elbow', 'Left Wrist', 'Right Wrist', 'Left Hip', 'Right Hip', 'Left Knee', 'Right Knee', 'Left Ankle', 'Right Ankle', 'Left Heel', 'Right Heel', 'Left Foot Index', 'Right Foot Index']
            fieldnames.extend([
                'Limb Straightness Left hand',
                'Limb Straightness Right hand',
                'Limb Straightness Left leg',
                'Limb Straightness Right leg',
                'Right hand Left leg coordination',
                'Left hand Right leg coordination',
                'Upper Body Straightness',
                'Body Straightness',
                'Distance from Saggital Plane',
                'Angle between feets'
            ])

            for i in range(num_points):
                fieldnames.append(f'CD{i}')

            for i in range(num_points):
                for j in range(i + 1, num_points):
                    fieldnames.append(f'MD({i},{j})')

            csv_writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            csv_writer.writeheader()
            frame_count = 0

            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = holistic.process(rgb_frame)

                if results.pose_world_landmarks:
                    X0 = (results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EYE].x + results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_EYE].x) / 2, (results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EYE].y + results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_EYE].y) / 2, (results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EYE].z + results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_EYE].z) / 2

                    X1 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER].z
                    X2 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].z

                    X3 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].z
                    X4 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].z

                    X5 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].z
                    X6 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].z

                    X7 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP].z
                    X8 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP].z

                    X9 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_KNEE].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_KNEE].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_KNEE].z
                    X10 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_KNEE].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_KNEE].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_KNEE].z

                    X11 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ANKLE].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ANKLE].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ANKLE].z
                    X12 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ANKLE].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ANKLE].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ANKLE].z

                    X13 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HEEL].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HEEL].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HEEL].z
                    X14 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HEEL].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HEEL].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HEEL].z

                    X15 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_FOOT_INDEX].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_FOOT_INDEX].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_FOOT_INDEX].z
                    X16 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_FOOT_INDEX].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_FOOT_INDEX].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_FOOT_INDEX].z

                    # Mutual distances calculation in Matrix form
                    coordinates = [X0, X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15, X16]
                    mutual_distances = [[0.0] * num_points for _ in range(num_points)]
                    for i in range(num_points):
                        for j in range(i + 1, num_points):
                            mutual_distance_ij = calculate_distance(coordinates[i], coordinates[j])
                            mutual_distances[i][j] = mutual_distances[j][i] = mutual_distance_ij

                    # Calculation of middle distances, as a tuple
                    Midjointsum = (X0 + X1 + X2 + X3 + X4 + X5 + X6 + X7 + X8 + X9 + X10 + X11 + X12 + X13 + X15 + X14 + X16)
                    midjoint = Midjointsum[0] / 17, Midjointsum[1] / 17, Midjointsum[2] / 17
                    C_D = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
                    for i in range(17):
                        C_D[i] = calculate_distance(midjoint, coordinates[i])

                    # Limb straightness calculation
                    num_rows = 4
                    num_cols = 3
                    variables = [[X1, X3, X5],
                                [X2, X4, X6],
                                [X7, X9, X11],
                                [X8, X10, X12]]
                    ls = [0, 0, 0, 0]
                    for i in range(num_rows):
                        Xa = variables[i][0]
                        Xb = variables[i][1]
                        Xc = variables[i][2]
                        ls[i] = pd_distance(Xa, Xb, Xc)

                    # Upper body straightness Calculations
                    Sm = (np.array(X1) + np.array(X2)) / 2
                    Hm = (np.array(X7) + np.array(X8)) / 2
                    US = pd_distance(X0, Hm, Sm)

                    # Body straightness Calculations
                    Am = (np.array(X11) + np.array(X12)) / 2
                    BS = pd_distance(Sm, Hm, Am)

                    # Calculation of angle between feets
                    AF = calculate_angle(X15, X13, X16, X14)

                    # Calculation of distance from saggital plane
                    CmHm = np.array(Hm) - np.array(X0)
                    SmHm = np.array(Hm) - np.array(Sm)
                    N = cross_p(CmHm, SmHm)
                    AlHm = np.array(X13) - np.array(Hm)
                    saggital_dist = np.dot(AlHm, N) / np.linalg.norm(N)

                    # Hand-leg coordination
                    hl_1 = calculate_angle(X2, X6, X7, X11)
                    hl_2 = calculate_angle(X1, X5, X8, X12)

                    # Write data to CSV
                    frame_data = {
                        'Frame': frame_count,
                        'Head': X0,
                        'Left Shoulder': X1,
                        'Right Shoulder': X2,
                        'Left Elbow': X3,
                        'Right Elbow': X4,
                        'Left Wrist': X5,
                        'Right Wrist': X6,
                        'Left Hip': X7,
                        'Right Hip': X8,
                        'Left Knee': X9,
                        'Right Knee': X10,
                        'Left Ankle': X11,
                        'Right Ankle': X12,
                        'Left Heel': X13,
                        'Right Heel': X14,
                        'Left Foot Index': X15,
                        'Right Foot Index': X16,
                        **{f'MD({i},{j})': mutual_distances[i][j] for i in range(num_points) for j in range(i + 1, num_points)},
                        **{f'CD{i}': C_D[i] for i in range(num_points)},
                        'Limb Straightness Left hand': ls[0],
                        'Limb Straightness Right hand': ls[1],
                        'Limb Straightness Left leg': ls[2],
                        'Limb Straightness Right leg': ls[3],
                        'Right hand Left leg coordination': hl_1,
                        'Left hand Right leg coordination': hl_2,
                        'Upper Body Straightness': US,
                        'Body Straightness': BS,
                        'Angle between feets': AF,
                        'Distance from Saggital Plane': saggital_dist
                    }
                    csv_writer.writerow(frame_data)
                frame_count += 1

        cap.release()

        print(f'3D Coordinates have been saved to {csv_file_path}')

/usr/local/lib/python3.10/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


3D Coordinates have been saved to /content/drive/MyDrive/surge/Frameworks/mediapipe/Everyvideoexceldata_mp/dip/2_10_1.csv
3D Coordinates have been saved to /content/drive/MyDrive/surge/Frameworks/mediapipe/Everyvideoexceldata_mp/dip/2_10_2.csv
3D Coordinates have been saved to /content/drive/MyDrive/surge/Frameworks/mediapipe/Everyvideoexceldata_mp/dip/2_11_1.csv
3D Coordinates have been saved to /content/drive/MyDrive/surge/Frameworks/mediapipe/Everyvideoexceldata_mp/dip/2_12_1.csv
3D Coordinates have been saved to /content/drive/MyDrive/surge/Frameworks/mediapipe/Everyvideoexceldata_mp/dip/2_13_1.csv
3D Coordinates have been saved to /content/drive/MyDrive/surge/Frameworks/mediapipe/Everyvideoexceldata_mp/dip/2_13_2.csv
3D Coordinates have been saved to /content/drive/MyDrive/surge/Frameworks/mediapipe/Everyvideoexceldata_mp/dip/2_14_1.csv
3D Coordinates have been saved to /content/drive/MyDrive/surge/Frameworks/mediapipe/Everyvideoexceldata_mp/dip/2_15_1.csv
3D Coordinates have been

In [ ]:
# Importing holistic class from MediaPipe
mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic(static_image_mode=False)

# Manually specify the paths
video_dir = '/content/drive/MyDrive/surge/Journal Dataset/hemiplegia'
output_dir = '/content/drive/MyDrive/surge/Frameworks/mediapipe/Everyvideoexceldata_mp/hemi'

# Ensure the paths are absolute
video_dir = os.path.abspath(video_dir)
output_dir = os.path.abspath(output_dir)

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for video_file in os.listdir(video_dir):
    if video_file.endswith(('.mp4', '.avi', '.mov')):  # Add other video file extensions if needed
        video_path = os.path.join(video_dir, video_file)
        cap = cv2.VideoCapture(video_path)

        csv_file_path = os.path.join(output_dir, f"{os.path.splitext(video_file)[0]}.csv")

        with open(csv_file_path, 'w', newline='') as csvfile:
            num_points = 17
            fieldnames = ['Frame', 'Head', 'Left Shoulder', 'Right Shoulder', 'Left Elbow', 'Right Elbow', 'Left Wrist', 'Right Wrist', 'Left Hip', 'Right Hip', 'Left Knee', 'Right Knee', 'Left Ankle', 'Right Ankle', 'Left Heel', 'Right Heel', 'Left Foot Index', 'Right Foot Index']
            fieldnames.extend([
                'Limb Straightness Left hand',
                'Limb Straightness Right hand',
                'Limb Straightness Left leg',
                'Limb Straightness Right leg',
                'Right hand Left leg coordination',
                'Left hand Right leg coordination',
                'Upper Body Straightness',
                'Body Straightness',
                'Distance from Saggital Plane',
                'Angle between feets'
            ])

            for i in range(num_points):
                fieldnames.append(f'CD{i}')

            for i in range(num_points):
                for j in range(i + 1, num_points):
                    fieldnames.append(f'MD({i},{j})')

            csv_writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            csv_writer.writeheader()
            frame_count = 0

            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = holistic.process(rgb_frame)

                if results.pose_world_landmarks:
                    X0 = (results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EYE].x + results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_EYE].x) / 2, (results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EYE].y + results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_EYE].y) / 2, (results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EYE].z + results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_EYE].z) / 2

                    X1 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER].z
                    X2 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].z

                    X3 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].z
                    X4 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].z

                    X5 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].z
                    X6 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].z

                    X7 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP].z
                    X8 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP].z

                    X9 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_KNEE].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_KNEE].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_KNEE].z
                    X10 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_KNEE].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_KNEE].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_KNEE].z

                    X11 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ANKLE].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ANKLE].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ANKLE].z
                    X12 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ANKLE].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ANKLE].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ANKLE].z

                    X13 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HEEL].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HEEL].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HEEL].z
                    X14 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HEEL].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HEEL].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HEEL].z

                    X15 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_FOOT_INDEX].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_FOOT_INDEX].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_FOOT_INDEX].z
                    X16 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_FOOT_INDEX].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_FOOT_INDEX].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_FOOT_INDEX].z

                    # Mutual distances calculation in Matrix form
                    coordinates = [X0, X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15, X16]
                    mutual_distances = [[0.0] * num_points for _ in range(num_points)]
                    for i in range(num_points):
                        for j in range(i + 1, num_points):
                            mutual_distance_ij = calculate_distance(coordinates[i], coordinates[j])
                            mutual_distances[i][j] = mutual_distances[j][i] = mutual_distance_ij

                    # Calculation of middle distances, as a tuple
                    Midjointsum = (X0 + X1 + X2 + X3 + X4 + X5 + X6 + X7 + X8 + X9 + X10 + X11 + X12 + X13 + X15 + X14 + X16)
                    midjoint = Midjointsum[0] / 17, Midjointsum[1] / 17, Midjointsum[2] / 17
                    C_D = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
                    for i in range(17):
                        C_D[i] = calculate_distance(midjoint, coordinates[i])

                    # Limb straightness calculation
                    num_rows = 4
                    num_cols = 3
                    variables = [[X1, X3, X5],
                                [X2, X4, X6],
                                [X7, X9, X11],
                                [X8, X10, X12]]
                    ls = [0, 0, 0, 0]
                    for i in range(num_rows):
                        Xa = variables[i][0]
                        Xb = variables[i][1]
                        Xc = variables[i][2]
                        ls[i] = pd_distance(Xa, Xb, Xc)

                    # Upper body straightness Calculations
                    Sm = (np.array(X1) + np.array(X2)) / 2
                    Hm = (np.array(X7) + np.array(X8)) / 2
                    US = pd_distance(X0, Hm, Sm)

                    # Body straightness Calculations
                    Am = (np.array(X11) + np.array(X12)) / 2
                    BS = pd_distance(Sm, Hm, Am)

                    # Calculation of angle between feets
                    AF = calculate_angle(X15, X13, X16, X14)

                    # Calculation of distance from saggital plane
                    CmHm = np.array(Hm) - np.array(X0)
                    SmHm = np.array(Hm) - np.array(Sm)
                    N = cross_p(CmHm, SmHm)
                    AlHm = np.array(X13) - np.array(Hm)
                    saggital_dist = np.dot(AlHm, N) / np.linalg.norm(N)

                    # Hand-leg coordination
                    hl_1 = calculate_angle(X2, X6, X7, X11)
                    hl_2 = calculate_angle(X1, X5, X8, X12)

                    # Write data to CSV
                    frame_data = {
                        'Frame': frame_count,
                        'Head': X0,
                        'Left Shoulder': X1,
                        'Right Shoulder': X2,
                        'Left Elbow': X3,
                        'Right Elbow': X4,
                        'Left Wrist': X5,
                        'Right Wrist': X6,
                        'Left Hip': X7,
                        'Right Hip': X8,
                        'Left Knee': X9,
                        'Right Knee': X10,
                        'Left Ankle': X11,
                        'Right Ankle': X12,
                        'Left Heel': X13,
                        'Right Heel': X14,
                        'Left Foot Index': X15,
                        'Right Foot Index': X16,
                        **{f'MD({i},{j})': mutual_distances[i][j] for i in range(num_points) for j in range(i + 1, num_points)},
                        **{f'CD{i}': C_D[i] for i in range(num_points)},
                        'Limb Straightness Left hand': ls[0],
                        'Limb Straightness Right hand': ls[1],
                        'Limb Straightness Left leg': ls[2],
                        'Limb Straightness Right leg': ls[3],
                        'Right hand Left leg coordination': hl_1,
                        'Left hand Right leg coordination': hl_2,
                        'Upper Body Straightness': US,
                        'Body Straightness': BS,
                        'Angle between feets': AF,
                        'Distance from Saggital Plane': saggital_dist
                    }
                    csv_writer.writerow(frame_data)
                frame_count += 1

        cap.release()

        print(f'3D Coordinates have been saved to {csv_file_path}')

In [ ]:
# Importing holistic class from MediaPipe
mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic(static_image_mode=False)

# Manually specify the paths
video_dir = '/content/drive/MyDrive/surge/Journal Dataset/normal'
output_dir = '/content/drive/MyDrive/surge/Frameworks/mediapipe/Everyvideoexceldata_mp/normal'

# Ensure the paths are absolute
video_dir = os.path.abspath(video_dir)
output_dir = os.path.abspath(output_dir)

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for video_file in os.listdir(video_dir):
    if video_file.endswith(('.mp4', '.avi', '.mov')):  # Add other video file extensions if needed
        video_path = os.path.join(video_dir, video_file)
        cap = cv2.VideoCapture(video_path)

        csv_file_path = os.path.join(output_dir, f"{os.path.splitext(video_file)[0]}.csv")

        with open(csv_file_path, 'w', newline='') as csvfile:
            num_points = 17
            fieldnames = ['Frame', 'Head', 'Left Shoulder', 'Right Shoulder', 'Left Elbow', 'Right Elbow', 'Left Wrist', 'Right Wrist', 'Left Hip', 'Right Hip', 'Left Knee', 'Right Knee', 'Left Ankle', 'Right Ankle', 'Left Heel', 'Right Heel', 'Left Foot Index', 'Right Foot Index']
            fieldnames.extend([
                'Limb Straightness Left hand',
                'Limb Straightness Right hand',
                'Limb Straightness Left leg',
                'Limb Straightness Right leg',
                'Right hand Left leg coordination',
                'Left hand Right leg coordination',
                'Upper Body Straightness',
                'Body Straightness',
                'Distance from Saggital Plane',
                'Angle between feets'
            ])

            for i in range(num_points):
                fieldnames.append(f'CD{i}')

            for i in range(num_points):
                for j in range(i + 1, num_points):
                    fieldnames.append(f'MD({i},{j})')

            csv_writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            csv_writer.writeheader()
            frame_count = 0

            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = holistic.process(rgb_frame)

                if results.pose_world_landmarks:
                    X0 = (results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EYE].x + results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_EYE].x) / 2, (results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EYE].y + results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_EYE].y) / 2, (results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EYE].z + results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_EYE].z) / 2

                    X1 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER].z
                    X2 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].z

                    X3 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].z
                    X4 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].z

                    X5 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].z
                    X6 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].z

                    X7 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP].z
                    X8 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP].z

                    X9 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_KNEE].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_KNEE].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_KNEE].z
                    X10 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_KNEE].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_KNEE].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_KNEE].z

                    X11 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ANKLE].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ANKLE].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ANKLE].z
                    X12 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ANKLE].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ANKLE].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ANKLE].z

                    X13 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HEEL].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HEEL].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HEEL].z
                    X14 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HEEL].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HEEL].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HEEL].z

                    X15 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_FOOT_INDEX].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_FOOT_INDEX].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_FOOT_INDEX].z
                    X16 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_FOOT_INDEX].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_FOOT_INDEX].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_FOOT_INDEX].z

                    # Mutual distances calculation in Matrix form
                    coordinates = [X0, X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15, X16]
                    mutual_distances = [[0.0] * num_points for _ in range(num_points)]
                    for i in range(num_points):
                        for j in range(i + 1, num_points):
                            mutual_distance_ij = calculate_distance(coordinates[i], coordinates[j])
                            mutual_distances[i][j] = mutual_distances[j][i] = mutual_distance_ij

                    # Calculation of middle distances, as a tuple
                    Midjointsum = (X0 + X1 + X2 + X3 + X4 + X5 + X6 + X7 + X8 + X9 + X10 + X11 + X12 + X13 + X15 + X14 + X16)
                    midjoint = Midjointsum[0] / 17, Midjointsum[1] / 17, Midjointsum[2] / 17
                    C_D = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
                    for i in range(17):
                        C_D[i] = calculate_distance(midjoint, coordinates[i])

                    # Limb straightness calculation
                    num_rows = 4
                    num_cols = 3
                    variables = [[X1, X3, X5],
                                [X2, X4, X6],
                                [X7, X9, X11],
                                [X8, X10, X12]]
                    ls = [0, 0, 0, 0]
                    for i in range(num_rows):
                        Xa = variables[i][0]
                        Xb = variables[i][1]
                        Xc = variables[i][2]
                        ls[i] = pd_distance(Xa, Xb, Xc)

                    # Upper body straightness Calculations
                    Sm = (np.array(X1) + np.array(X2)) / 2
                    Hm = (np.array(X7) + np.array(X8)) / 2
                    US = pd_distance(X0, Hm, Sm)

                    # Body straightness Calculations
                    Am = (np.array(X11) + np.array(X12)) / 2
                    BS = pd_distance(Sm, Hm, Am)

                    # Calculation of angle between feets
                    AF = calculate_angle(X15, X13, X16, X14)

                    # Calculation of distance from saggital plane
                    CmHm = np.array(Hm) - np.array(X0)
                    SmHm = np.array(Hm) - np.array(Sm)
                    N = cross_p(CmHm, SmHm)
                    AlHm = np.array(X13) - np.array(Hm)
                    saggital_dist = np.dot(AlHm, N) / np.linalg.norm(N)

                    # Hand-leg coordination
                    hl_1 = calculate_angle(X2, X6, X7, X11)
                    hl_2 = calculate_angle(X1, X5, X8, X12)

                    # Write data to CSV
                    frame_data = {
                        'Frame': frame_count,
                        'Head': X0,
                        'Left Shoulder': X1,
                        'Right Shoulder': X2,
                        'Left Elbow': X3,
                        'Right Elbow': X4,
                        'Left Wrist': X5,
                        'Right Wrist': X6,
                        'Left Hip': X7,
                        'Right Hip': X8,
                        'Left Knee': X9,
                        'Right Knee': X10,
                        'Left Ankle': X11,
                        'Right Ankle': X12,
                        'Left Heel': X13,
                        'Right Heel': X14,
                        'Left Foot Index': X15,
                        'Right Foot Index': X16,
                        **{f'MD({i},{j})': mutual_distances[i][j] for i in range(num_points) for j in range(i + 1, num_points)},
                        **{f'CD{i}': C_D[i] for i in range(num_points)},
                        'Limb Straightness Left hand': ls[0],
                        'Limb Straightness Right hand': ls[1],
                        'Limb Straightness Left leg': ls[2],
                        'Limb Straightness Right leg': ls[3],
                        'Right hand Left leg coordination': hl_1,
                        'Left hand Right leg coordination': hl_2,
                        'Upper Body Straightness': US,
                        'Body Straightness': BS,
                        'Angle between feets': AF,
                        'Distance from Saggital Plane': saggital_dist
                    }
                    csv_writer.writerow(frame_data)
                frame_count += 1

        cap.release()

        print(f'3D Coordinates have been saved to {csv_file_path}')

In [ ]:
# Importing holistic class from MediaPipe
mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic(static_image_mode=False)

# Manually specify the paths
video_dir = '/content/drive/MyDrive/surge/Journal Dataset/parkinson'
output_dir = '/content/drive/MyDrive/surge/Frameworks/mediapipe/Everyvideoexceldata_mp/parkinson'

# Ensure the paths are absolute
video_dir = os.path.abspath(video_dir)
output_dir = os.path.abspath(output_dir)

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for video_file in os.listdir(video_dir):
    if video_file.endswith(('.mp4', '.avi', '.mov')):  # Add other video file extensions if needed
        video_path = os.path.join(video_dir, video_file)
        cap = cv2.VideoCapture(video_path)

        csv_file_path = os.path.join(output_dir, f"{os.path.splitext(video_file)[0]}.csv")

        with open(csv_file_path, 'w', newline='') as csvfile:
            num_points = 17
            fieldnames = ['Frame', 'Head', 'Left Shoulder', 'Right Shoulder', 'Left Elbow', 'Right Elbow', 'Left Wrist', 'Right Wrist', 'Left Hip', 'Right Hip', 'Left Knee', 'Right Knee', 'Left Ankle', 'Right Ankle', 'Left Heel', 'Right Heel', 'Left Foot Index', 'Right Foot Index']
            fieldnames.extend([
                'Limb Straightness Left hand',
                'Limb Straightness Right hand',
                'Limb Straightness Left leg',
                'Limb Straightness Right leg',
                'Right hand Left leg coordination',
                'Left hand Right leg coordination',
                'Upper Body Straightness',
                'Body Straightness',
                'Distance from Saggital Plane',
                'Angle between feets'
            ])

            for i in range(num_points):
                fieldnames.append(f'CD{i}')

            for i in range(num_points):
                for j in range(i + 1, num_points):
                    fieldnames.append(f'MD({i},{j})')

            csv_writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            csv_writer.writeheader()
            frame_count = 0

            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = holistic.process(rgb_frame)

                if results.pose_world_landmarks:
                    X0 = (results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EYE].x + results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_EYE].x) / 2, (results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EYE].y + results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_EYE].y) / 2, (results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EYE].z + results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_EYE].z) / 2

                    X1 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER].z
                    X2 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].z

                    X3 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].z
                    X4 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].z

                    X5 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].z
                    X6 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].z

                    X7 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP].z
                    X8 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP].z

                    X9 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_KNEE].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_KNEE].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_KNEE].z
                    X10 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_KNEE].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_KNEE].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_KNEE].z

                    X11 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ANKLE].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ANKLE].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ANKLE].z
                    X12 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ANKLE].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ANKLE].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ANKLE].z

                    X13 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HEEL].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HEEL].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HEEL].z
                    X14 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HEEL].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HEEL].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HEEL].z

                    X15 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_FOOT_INDEX].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_FOOT_INDEX].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_FOOT_INDEX].z
                    X16 = results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_FOOT_INDEX].x, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_FOOT_INDEX].y, results.pose_world_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_FOOT_INDEX].z

                    # Mutual distances calculation in Matrix form
                    coordinates = [X0, X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15, X16]
                    mutual_distances = [[0.0] * num_points for _ in range(num_points)]
                    for i in range(num_points):
                        for j in range(i + 1, num_points):
                            mutual_distance_ij = calculate_distance(coordinates[i], coordinates[j])
                            mutual_distances[i][j] = mutual_distances[j][i] = mutual_distance_ij

                    # Calculation of middle distances, as a tuple
                    Midjointsum = (X0 + X1 + X2 + X3 + X4 + X5 + X6 + X7 + X8 + X9 + X10 + X11 + X12 + X13 + X15 + X14 + X16)
                    midjoint = Midjointsum[0] / 17, Midjointsum[1] / 17, Midjointsum[2] / 17
                    C_D = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
                    for i in range(17):
                        C_D[i] = calculate_distance(midjoint, coordinates[i])

                    # Limb straightness calculation
                    num_rows = 4
                    num_cols = 3
                    variables = [[X1, X3, X5],
                                [X2, X4, X6],
                                [X7, X9, X11],
                                [X8, X10, X12]]
                    ls = [0, 0, 0, 0]
                    for i in range(num_rows):
                        Xa = variables[i][0]
                        Xb = variables[i][1]
                        Xc = variables[i][2]
                        ls[i] = pd_distance(Xa, Xb, Xc)

                    # Upper body straightness Calculations
                    Sm = (np.array(X1) + np.array(X2)) / 2
                    Hm = (np.array(X7) + np.array(X8)) / 2
                    US = pd_distance(X0, Hm, Sm)

                    # Body straightness Calculations
                    Am = (np.array(X11) + np.array(X12)) / 2
                    BS = pd_distance(Sm, Hm, Am)

                    # Calculation of angle between feets
                    AF = calculate_angle(X15, X13, X16, X14)

                    # Calculation of distance from saggital plane
                    CmHm = np.array(Hm) - np.array(X0)
                    SmHm = np.array(Hm) - np.array(Sm)
                    N = cross_p(CmHm, SmHm)
                    AlHm = np.array(X13) - np.array(Hm)
                    saggital_dist = np.dot(AlHm, N) / np.linalg.norm(N)

                    # Hand-leg coordination
                    hl_1 = calculate_angle(X2, X6, X7, X11)
                    hl_2 = calculate_angle(X1, X5, X8, X12)

                    # Write data to CSV
                    frame_data = {
                        'Frame': frame_count,
                        'Head': X0,
                        'Left Shoulder': X1,
                        'Right Shoulder': X2,
                        'Left Elbow': X3,
                        'Right Elbow': X4,
                        'Left Wrist': X5,
                        'Right Wrist': X6,
                        'Left Hip': X7,
                        'Right Hip': X8,
                        'Left Knee': X9,
                        'Right Knee': X10,
                        'Left Ankle': X11,
                        'Right Ankle': X12,
                        'Left Heel': X13,
                        'Right Heel': X14,
                        'Left Foot Index': X15,
                        'Right Foot Index': X16,
                        **{f'MD({i},{j})': mutual_distances[i][j] for i in range(num_points) for j in range(i + 1, num_points)},
                        **{f'CD{i}': C_D[i] for i in range(num_points)},
                        'Limb Straightness Left hand': ls[0],
                        'Limb Straightness Right hand': ls[1],
                        'Limb Straightness Left leg': ls[2],
                        'Limb Straightness Right leg': ls[3],
                        'Right hand Left leg coordination': hl_1,
                        'Left hand Right leg coordination': hl_2,
                        'Upper Body Straightness': US,
                        'Body Straightness': BS,
                        'Angle between feets': AF,
                        'Distance from Saggital Plane': saggital_dist
                    }
                    csv_writer.writerow(frame_data)
                frame_count += 1

        cap.release()

        print(f'3D Coordinates have been saved to {csv_file_path}')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import os

def extract_averages(input_folder, output_file):
    # Dictionary to store column sums and counts for each CSV file
    file_column_sums = {}
    file_column_counts = {}

    # Iterate over each CSV file in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".csv"):
            # Read CSV file into a DataFrame
            filepath = os.path.join(input_folder, filename)
            df = pd.read_csv(filepath)

            # Extract data from columns S to FY starting from the second row
            data = df.iloc[1:, 18:182]  # Assuming S is the 19th column and FY is the 181st column

            # Convert data to numeric
            numeric_data = data.apply(pd.to_numeric, errors='coerce')

            # Initialize column sums and counts for this file
            file_column_sums[filename] = {}
            file_column_counts[filename] = {}

            # Update column sums and counts for this file
            for column in numeric_data:
                column_data = numeric_data[column]
                column_sum = column_data.sum()
                column_count = column_data.count()
                file_column_sums[filename][column] = column_sum
                file_column_counts[filename][column] = column_count

    # Create a DataFrame to store averages for each CSV file
    file_averages_dfs = []
    for filename in file_column_sums:
        # Calculate column averages for this file
        file_column_averages = {column: file_column_sums[filename][column] / file_column_counts[filename][column]
                                for column in file_column_sums[filename]}
        file_averages_df = pd.DataFrame.from_dict(file_column_averages, orient='index', columns=[filename])
        file_averages_dfs.append(file_averages_df)

    combined_df = pd.concat(file_averages_dfs, axis=1)

    combined_df.to_csv(output_file)

input_folder = '/content/drive/MyDrive/surge'
output_file = 'column_averages4.csv'
extract_averages(input_folder, output_file)